# Case: Criminal location tracking
**Context**
> *During several home searches by the Dutch police, phones were confiscated. After examining these phones, they were found to contain GPS information. It is believed that these two individuals met. Given the importance of the criminal investigation, can you find out the exact spot where these two individuals met?*

<img src="../img/banner_case.png" width=600 display=block margin-left=auto, margin-right: auto>

The solutions is broken down into the following steps:
1. Read in the location data
2. Visualize the route of the two criminals on a map
3. Get the exact times and coordinates when/where they met


<br><details><summary>Hints</summary>

- Sometimes one criminal is standing still for a while, when the other one is moving!
- Animate the route of the criminals during the day, to see where they are at what time

</div>


<small> source: [https://www.crimediggers.nl/]() </small>


In [ ]:
# Import libraries
import pandas as pd
import numpy as np

## 1. Read in the location data
- check which columns are in it, and the data types
- convert the 'datetime' column to datetime type
- check if there are any missing values
- order the data chronologically with `sort_values`

In [ ]:
# Read in the data in the /data/ folder
#gps1 =
#gps2 = 

# Check the columns and data types


# Check for missing values


# Sort the data chronologically


# Output the first 5 rows

## 2. Visualize the route of the two criminals on a map
For data visualisation, the `plotly.express` libary is a great choice. It is a high-level library that is widely used. You can find documentation [here](https://plotly.com/python/).


<br><details><summary>Hint 1</summary>

- use `px.scatter_mapbox()` to create a map
- check the parameters of `px.scatter_mapbox()` function to customize your map
</details> 

</div>

<br><details><summary>Hint 2</summary>

- You can animate the map by adding the parameter `animation_frame` to the `px.scatter_mapbox()` function

</div>

In [ ]:
# Import libraries for visualisation
import plotly.express as px
import plotly.graph_objects as go

### Criminal 1
Create a map of the route of criminal 1 in Amsterdam.

<br><details><summary>Hint</summary>
    - use `.dt.time` to get the extract only time from the datetime column

In [ ]:
# Add time column
#gps1['time'] = 
#gps2['time'] =

# Create an animated map using Plotly

#fig = 

# Customize the appearance of the map if needed
#fig.update_layout()

#fig.show()

### Criminal 2
Copy your code from the previous cell and change the dataframe to `gps2` to plot the route of criminal 2.

In [ ]:
#fig 

# Customize the appearance of the map if needed

# Change trace color

#fig.show()

### Plot route of both criminals on one map to see which places they have in common. 

<br><details><summary>Hint</summary>
    - add a new column to each dataframe with the name of the criminal
    - use `pd.concat()` to merge the two dataframes
    - use `px.line_mapbox()` to create a map
    
</div>

<br><details><summary>Hint</summary>
    - use the `color` parameter to color the routes differently
    
</div>

You will see that the two criminals share common places. But when where they at the same place **at the same time?** You can manually check for each point, but that is not very efficient. In section 3, we will try to find the exact time and place where they met.

In [ ]:
# Add a column with the name of the criminal

# Take the two dataframes together in one dataframe

# Sort values chronologically in time

# Visualize both routes on one map using the 'color' argument
#fig = px.line_mapbox()

## 3. Get the exact time and coordinates when they met
<br><details><summary>Hint</summary>
    - To calculate the distance between two gps points, we can use the `geodesic` function of the geopy library for this: https://geopy.readthedocs.io/en/stable/#geopy.distance.geodesic
    - use the `merge_ordered` function of pandas to take the gps data together of both suspects together. It merges the two gps location data together on the datetime column. The fill_method parameter of the function ensures that the last known location of suspect 2 is put together with the location of suspect 1. 
</details>

<br><details><summary>Hint 2</summary>
    - use the `min()` function to get the point in time when the two criminals were closest to each other
</details>

In [ ]:
# Calculate the distance between the two criminals for each point in time
from geopy.distance import geodesic

# Sort gps1 and gps2 values chronologically in time

# We want to use the `merge_ordered` function of pandas to take the gps data together of both suspects together. It merges the two gps location data together on the datetime column. The fill_method parameter of the function ensures that the last known location of suspect 2 is put together with the location of suspect 1. 
#gps_merged = pd.merge_ordered(gps1, gps2, on='datetime', fill_method='ffill', suffixes=['_suspect_1', '_suspect_2'])
#gps_merged[['longitude_suspect_1', 'latitude_suspect_1', 'longitude_suspect_2', 'latitude_suspect_2']] = gps_merged[['longitude_suspect_1', 'latitude_suspect_1', 'longitude_suspect_2', 'latitude_suspect_2']].fillna(0)

# Add a new column to the merged dataframe with the distance between the two criminals
# Use the `geodesic` function of the geopy library for this: https://geopy.readthedocs.io/en/stable/#module-geopy.distance
#gps_merged['distance'] = gps_merged.apply(lambda x: geodesic((x['latitude_suspect_1'], x['longitude_suspect_1']), (x['latitude_suspect_2'], x['longitude_suspect_2'])).meters, axis=1)

In [ ]:
# Get point in time when the two criminals were closest to each other
# meeting_point = gps_merged[gps_merged['distance'] == gps_merged['distance'].min()]


In [ ]:
# Plot the meeting point on a map
#fig = px.scatter_mapbox()